# Bundestagsscraper:

## Import & Setup

In [ ]:
#This is likely necessary to run the scripts. You can go ahead and try to run deach script with out installing requirements.txt as there might be some redundant packages or you have them installed already
%pip install -r requirements.txt

In [ ]:

import urllib.request, urllib.error, urllib.parse
import regex as re
from bs4 import BeautifulSoup
import time
import pandas as pd

apiKey = "Your API Key Here"

Set your timerange here. Honestly very clunky, there will be a rework of this scraper soon

In [2]:
#AnfangArr = [1950,3,1]
AnfangArr = [2025,1,1]

startDatum = str(AnfangArr[0]) + "-" + str(AnfangArr[1]) + "-" + str(AnfangArr[2])
startDatum = re.sub(r'(?<!\d)(\d)(?!\d)', r'0\1', startDatum)

#EndeArr = [1950,3,18]
EndeArr = [2025,10,2]
endDatum = str(EndeArr[0]) + "-" + str(EndeArr[1]) + "-" + str(EndeArr[2])
endDatum = re.sub(r'(?<!\d)(\d)(?!\d)', r'0\1', endDatum)

## Download Funktion

## Haupt-Schleife

In [7]:
KurzURL = "https://search.dip.bundestag.de/api/v1/plenarprotokoll?f.zuordnung=BT&f.datum.start="+startDatum+"&f.datum.end="+endDatum+"&format=xml&apikey="+apiKey

print(KurzURL)

https://search.dip.bundestag.de/api/v1/plenarprotokoll?f.zuordnung=BT&f.datum.start=2025-01-02&f.datum.end=2025-15-02&format=xml&apikey=OSOegLs.PR2lwJ1dwCeje9vTj7FPOt3hvpYKtwKkhw


Test this URL to see if things still work

In [ ]:
def requester(AnfangArr, EndeArr):
    startDatum = str(AnfangArr[0]) + "-" + str(AnfangArr[1]) + "-" + str(AnfangArr[2])
    startDatum = re.sub(r'(?<!\d)(\d)(?!\d)', r'0\1', startDatum)

    endDatum = str(EndeArr[0]) + "-" + str(EndeArr[1]) + "-" + str(EndeArr[2])
    endDatum = re.sub(r'(?<!\d)(\d)(?!\d)', r'0\1', endDatum)

    KurzURL = "https://search.dip.bundestag.de/api/v1/plenarprotokoll?f.zuordnung=BT&f.datum.start="+startDatum+"&f.datum.end="+endDatum+"&format=json&apikey="+apiKey
    print(KurzURL)
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    req = urllib.request.Request(KurzURL, headers=headers)
    response = urllib.request.urlopen(req)
    webTextInhalt = response.read().decode('UTF-8')

    keyword = "documents"
    before_keyword, keyword, after_keyword = webTextInhalt.partition(keyword)

        #Die BundestagsAPI erlaubt nur 10 Dokumente pro Anfrage, daher wird die Anfrage aufgeteilt in 10 Dokumente pro Anfrage. Ndocs erhebt die Nummer der Dokumente innerhalb der Suchreichweite und teilt ggf die Suchreichweite auf.
    nDocs = re.findall(r'\d+', before_keyword)
    Docs = int(nDocs[0])
        
    if Docs >= 10:
        print("Zu viele Dokumente, verringere Suchbereich")

        if((AnfangArr[0] == EndeArr[0]) and (AnfangArr[2] >= 15) and (EndeArr[2] == 1)) or ((AnfangArr[0] == EndeArr[0]) and (AnfangArr[2] >= 1) and (EndeArr[2] == 15) and (AnfangArr[1] == EndeArr[1])):
            print("Suchbereich zu klein")
            AnfangArr[2] = AnfangArr[2] +1

        elif (AnfangArr[0] == EndeArr[0]) and (AnfangArr[1] == EndeArr[1]) and (AnfangArr[2] == EndeArr[2]):
            print("Suchbereich zu klein")
            AnfangArr[0] = AnfangArr[0] -1


        elif (AnfangArr[1] >= EndeArr[1]) and (AnfangArr[1] < 12) and (AnfangArr[0] < EndeArr[0]):
            print("Monat zu klein")
            AnfangArr[1] = AnfangArr[1] + 1
         

        elif (AnfangArr[1] == 12 and EndeArr[1] != 1):
            print("Jahresende")
            AnfangArr[0] = AnfangArr[0] +1
            AnfangArr[1] = 1
           
        elif (AnfangArr[1] == 12 and EndeArr[1] == 1):
            print("Jahresende mit Jahresanfangskomplöxitäten")
            AnfangArr[2] = AnfangArr[2] + 1
           
        elif (AnfangArr[1] == (EndeArr[1]-1) and (AnfangArr[0] == EndeArr[0])) and (EndeArr[2] != 15):
            print("Mehr als 10 Einträge in einem Monat")
            AnfangArr[2] = 15

        elif (AnfangArr[1] == (EndeArr[1]-1) and (AnfangArr[0] == EndeArr[0])) and (EndeArr[2] == 15):
            print("Mehr als 10 Einträge in einem Monat")
            AnfangArr[2] = 1
            AnfangArr[1] = EndeArr[1]
        

        elif ((AnfangArr[1] == 12) and (AnfangArr[2] == 15)):
            AnfangArr[0] = AnfangArr[0] + 1
            AnfangArr[1] = 1
            AnfangArr[2] = 1


        else:
            print("Iteriere Monat")
            AnfangArr[1] = AnfangArr[1] + 1
         
        AnfangStr = str(AnfangArr[0]) + "-" + str(AnfangArr[1]) + "-" + str(AnfangArr[2]) #Die API erlaubt nur Datumsanfragen im Format YYYY-MM-DD daher wird das Datum in ein String umgewandelt und eine 0 vor einstelligte Tage oder Monate gesetzt. 
        return AnfangArr




    else:
        print("N < 10. Schreibe in Datei")
        TextURL = "https://search.dip.bundestag.de/api/v1/plenarprotokoll-text?f.zuordnung=BT&f.datum.start="+startDatum+"&f.datum.end="+endDatum+"&format=json&apikey="+apiKey
        print(TextURL)
        response = urllib.request.urlopen(TextURL)
        TextInhalt = response.read().decode('UTF-8')
                
        fileName = "../Speeches/TextDaten"+startDatum +"-"+ endDatum + ".txt" 
        file = open(fileName, "w")
        a = file.write(TextInhalt)
        file.close() #Datei abspeichern
        EndeArr[:] = [0, 0, 0]  
        
        print("Datei geschrieben")

        EndeArr[0] = AnfangArr[0]   #Neue Suchparameter definieren
        EndeArr[1] = AnfangArr[1]
        EndeArr[2] = AnfangArr[2]
        endDatum = str(EndeArr[0]) + "-" + str(EndeArr[1]) + "-" + str(EndeArr[2])
        endDatum = re.sub(r'(?<!\d)(\d)(?!\d)', r'0\1', endDatum)
        
        AnfangArr[0] = AnfangArr[0] -1
        AnfangArr[2] = 1
        return AnfangArr, EndeArr 



You need to create a TextDaten folder one layer above your scripts. Inside I recommend also creating a IndividualSpeeches folder. Otherwise my stuff will break

In [ ]:
while(AnfangArr[0] >= 1948 ):
    try:
        requester(AnfangArr, EndeArr)
        time.sleep(5)
    except:
        print("Fehler")
        time.sleep(600)